In [ ]:
DATA_DIR = '/home/pal.balazs/github/gizmo-public/ICs/'

In [ ]:
import os
import h5py

import numpy as np
from scipy.interpolate import griddata

import matplotlib.pyplot as plt

In [ ]:
def get_keys(s):
    with h5py.File(os.path.join(DATA_DIR, s), "r") as f:
        keys = list(f['PartType0'].keys())
    return keys

In [ ]:
def get_data(s):
    values = {}
    with h5py.File(os.path.join(DATA_DIR, s), "r") as f:
        for k in get_keys(s):
            values[k] = f['PartType0'][k][:]
    return values

In [ ]:
def get_grid_data(s, N):
    X, Y, D, _ = get_data(s)
    xi, yi = np.meshgrid(np.linspace(0, 1, N), np.linspace(0, 1, N),
                         indexing='ij')
    return griddata(np.stack((X, Y), axis=1), D, (xi, yi), method='cubic')

## Generate initial conditions with smooth boundaries

In [ ]:
N  = 784   # Number of particles in a single dimension
L  = 1.0   # Size of the box
dh = 0.05  # Smoothing length of V and D on the fluid boundary

### Determine smoothing length

In [ ]:
delta_x = (L**2 / N**2)**0.5

### Generate coordinates

In [ ]:
# Uniformly random coordinates
coordinates = np.column_stack((
    np.random.uniform(low=[-0.5, -0.5], high=[0.5, 0.5], size=(N**2, 2)),
    np.zeros(N**2)
))

In [ ]:
# Coordinates on a grid 
xi = np.linspace(-0.5+delta_x, 0.5-delta_x, N)
yi = np.linspace(-0.5+delta_x, 0.5-delta_x, N)
X, Y = np.meshgrid(xi, yi, sparse=False, indexing='xy')
Z    = np.zeros(N**2)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.axis(False)
ax.scatter(X, Y, s=0.1, alpha=1.0)
plt.show()

### Generate densities

In [ ]:
rho_1, rho_2 = 1, 2  # Densities of the two interacting fluids

In [ ]:
D = np.tanh(Y/dh)
D = (D - np.min(D)) * (1 / (np.max(D) - np.min(D)))
D = D * np.abs(rho_1 - rho_2) + np.min((rho_1, rho_2))
D = D[::-1]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.axis(False)
ax.scatter(X, Y, c=D, s=0.1, alpha=1.0)
plt.show()

### Generate X-velocities

In [ ]:
vx = np.tanh(Y/dh) * 0.5

In [ ]:
vx

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.axis(False)
ax.scatter(X, Y, c=vx, s=0.1, alpha=1.0)
plt.show()

### Generate Y-velocities

In [ ]:
# Similarly to McNally et. al (2012)
vy = 0.01 * np.sin(4 * np.pi * X)

In [ ]:
vy

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.axis(False)
ax.scatter(X, Y, c=vy, s=0.1, alpha=1.0)
plt.show()

### Calculate internal energy

In [ ]:
P = 5./2.      # Constant pressure from McNally et al. (2012)
gamma = 5./3.  # Polytropic index of ideal EoS

In [ ]:
E = P / ((gamma - 1) * D)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.axis(False)
ax.scatter(X, Y, c=E, s=0.1, alpha=1.0)
plt.show()

### Calculate masses

In [ ]:
M = D / D.size

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_aspect('equal')
ax.axis(False)
ax.scatter(X, Y, c=M, s=0.1, alpha=1.0)
plt.show()

## Save initial condition file

In [ ]:
file = h5py.File(os.path.join(DATA_DIR, 'test_kh_ics.hdf5'), 'w')

h = file.create_group('Header')
NPART = np.array([N**2, 0, 0, 0, 0, 0])
h.attrs['NumPart_ThisFile'] = NPART
h.attrs['NumPart_Total'] = NPART
h.attrs['NumPart_Total_HighWord'] = 0*NPART
h.attrs['MassTable'] = np.zeros(6)
h.attrs['Time'] = 0.0
h.attrs['NumFilesPerSnapshot'] = 1
h.attrs['Flag_DoublePrecision'] = 0

p = file.create_group('PartType0')
p.create_dataset('Coordinates', data=np.column_stack((X.flatten(), Y.flatten(), Z.flatten())))
p.create_dataset('Velocities', data=np.column_stack((vx.flatten(), vy.flatten(), np.zeros(N**2))))
p.create_dataset('ParticleIDs', data=np.arange(1, N**2+1, 1))
p.create_dataset('ParticleChildIDsNumber', data=np.zeros(N**2))
p.create_dataset('ParticleIDGenerationNumber', data=np.zeros(N**2))
p.create_dataset('Masses', data=M.flatten())
p.create_dataset('Density', data=D.flatten())
p.create_dataset('InternalEnergy', data=E.flatten())
p.create_dataset('SmoothingLength', data=((M / D)**(1/2)).flatten())

file.close()

In [ ]:
s = os.path.join(DATA_DIR, 'test_kh_ics.hdf5')

In [ ]:
values = get_data(s)

In [ ]:
values

In [ ]:
idx = np.array([1, 5777, 9605, 22281]) - 1
for i in idx:
    print(f'{i+1}\t-- x: {X.flatten()[i]:.4f}\ty: {Y.flatten()[i]:.4f}\tvx: {V.flatten()[i]:.4f}\td: {D.flatten()[i]:.4f}')

In [ ]:
s = os.path.join('/home/pal.balazs/github/gizmo-public/scripts', 'box_3d_r32.hdf5')